<a href="https://colab.research.google.com/github/https-deeplearning-ai/tensorflow-3-public/blob/main/Course%201%20-%20Custom%20Models%2C%20Layers%20and%20Loss%20Functions/Week%205%20-%20Callbacks/C1_W5_Lab_1_exploring-callbacks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ungraded Lab: Introduction to Keras callbacks

In Keras, `Callback` is a Python class meant to be subclassed to provide specific functionality, with a set of methods called at various stages of training (including batch/epoch start and ends), testing, and predicting. Callbacks are useful to get a view on internal states and statistics of the model during training. The methods of the callbacks can  be called at different stages of training/evaluating/inference. Keras has available [callbacks](https://keras.io/api/callbacks/) and we'll show how you can use it in the following sections. Please click the **Open in Colab** badge above to complete this exercise in Colab. This will allow you to take advantage of the free GPU runtime (for faster training) and compatibility with all the packages needed in this notebook.

## Model methods that take callbacks
Users can supply a list of callbacks to the following `tf.keras.Model` methods:
* [`fit()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit), [`fit_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit_generator)
Trains the model for a fixed number of epochs (iterations over a dataset, or data yielded batch-by-batch by a Python generator).
* [`evaluate()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate), [`evaluate_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate_generator)
Evaluates the model for given data or data generator. Outputs the loss and metric values from the evaluation.
* [`predict()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict), [`predict_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict_generator)
Generates output predictions for the input data or data generator.

## Imports

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import io
from PIL import Image

from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, LearningRateScheduler, ModelCheckpoint, CSVLogger, ReduceLROnPlateau
%load_ext tensorboard

import os
import matplotlib.pylab as plt
import numpy as np
import math
import datetime
import pandas as pd

print("Version: ", tf.__version__)
tf.get_logger().setLevel('INFO')

Version:  2.1.0


# Examples of Keras callback applications
The following section will guide you through creating simple [Callback](https://keras.io/api/callbacks/) applications.

In [2]:
# Download and prepare the horses or humans dataset

# horses_or_humans 3.0.0 has already been downloaded for you
path = "./tensorflow_datasets"
splits, info = tfds.load('horses_or_humans', data_dir=path, as_supervised=True, with_info=True, split=['train[:80%]', 'train[80%:]', 'test'])

(train_examples, validation_examples, test_examples) = splits

num_examples = info.splits['train'].num_examples
num_classes = info.features['label'].num_classes

In [3]:
SIZE = 150 #@param {type:"slider", min:64, max:300, step:1}
IMAGE_SIZE = (SIZE, SIZE)

In [4]:
def format_image(image, label):
  image = tf.image.resize(image, IMAGE_SIZE) / 255.0
  return  image, label

In [5]:
BATCH_SIZE = 32 #@param {type:"integer"}

In [6]:
train_batches = train_examples.shuffle(num_examples // 4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_examples.map(format_image).batch(BATCH_SIZE).prefetch(1)
test_batches = test_examples.map(format_image).batch(1)

In [7]:
for image_batch, label_batch in train_batches.take(1):
  pass

image_batch.shape

TensorShape([32, 150, 150, 3])

In [8]:
def build_model(dense_units, input_shape=IMAGE_SIZE + (3,)):
  model = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=input_shape),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(dense_units, activation='relu'),
      tf.keras.layers.Dense(2, activation='softmax')
  ])
  return model

## [TensorBoard](https://keras.io/api/callbacks/tensorboard/)

Enable visualizations for TensorBoard.

In [9]:
!rm -rf logs

In [10]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir)

model.fit(train_batches, 
          epochs=10, 
          validation_data=validation_batches, 
          callbacks=[tensorboard_callback])

Epoch 1/10
26/26 [==============================] - 21s 806ms/step - loss: 0.6726 - accuracy: 0.5766 - val_loss: 0.6553 - val_accuracy: 0.6634
Epoch 2/10
26/26 [==============================] - 20s 776ms/step - loss: 0.6264 - accuracy: 0.6825 - val_loss: 0.6055 - val_accuracy: 0.6098
Epoch 3/10
26/26 [==============================] - 20s 766ms/step - loss: 0.5749 - accuracy: 0.7384 - val_loss: 0.5286 - val_accuracy: 0.8293
Epoch 4/10
26/26 [==============================] - 20s 765ms/step - loss: 0.5339 - accuracy: 0.7652 - val_loss: 0.5015 - val_accuracy: 0.8098
Epoch 5/10
26/26 [==============================] - 20s 762ms/step - loss: 0.4398 - accuracy: 0.8333 - val_loss: 0.3940 - val_accuracy: 0.8829
Epoch 6/10
26/26 [==============================] - 20s 769ms/step - loss: 0.4052 - accuracy: 0.8455 - val_loss: 0.3535 - val_accuracy: 0.8927
Epoch 7/10
26/26 [==============================] - 20s 769ms/step - loss: 0.3169 - accuracy: 0.8917 - val_loss: 0.2655 - val_accuracy: 0.9317

In [11]:
%tensorboard --logdir logs

## [Model Checkpoint](https://keras.io/api/callbacks/model_checkpoint/)

Callback to save the Keras model or model weights at some frequency.

In [12]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('weights.{epoch:02d}-{val_loss:.2f}.h5', verbose=1),
          ])

Epoch 1/5

Epoch 00001: saving model to weights.01-0.66.h5
26/26 - 21s - loss: 0.6783 - accuracy: 0.5523 - val_loss: 0.6628 - val_accuracy: 0.5463
Epoch 2/5

Epoch 00002: saving model to weights.02-0.65.h5
26/26 - 20s - loss: 0.6317 - accuracy: 0.6618 - val_loss: 0.6488 - val_accuracy: 0.5512
Epoch 3/5

Epoch 00003: saving model to weights.03-0.65.h5
26/26 - 20s - loss: 0.5945 - accuracy: 0.7007 - val_loss: 0.6530 - val_accuracy: 0.5610
Epoch 4/5

Epoch 00004: saving model to weights.04-0.53.h5
26/26 - 20s - loss: 0.5668 - accuracy: 0.7153 - val_loss: 0.5252 - val_accuracy: 0.7805
Epoch 5/5

Epoch 00005: saving model to weights.05-0.53.h5
26/26 - 20s - loss: 0.5162 - accuracy: 0.7628 - val_loss: 0.5299 - val_accuracy: 0.7122


In [13]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=1, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('saved_model', verbose=1)
          ])


Epoch 00001: saving model to saved_model
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


INFO:tensorflow:Assets written to: saved_model/assets


INFO:tensorflow:Assets written to: saved_model/assets


26/26 - 21s - loss: 0.6756 - accuracy: 0.5791 - val_loss: 0.6813 - val_accuracy: 0.4634


In [14]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=2, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('model.h5', verbose=1)
          ])

Epoch 1/2

Epoch 00001: saving model to model.h5
26/26 - 20s - loss: 0.6755 - accuracy: 0.5693 - val_loss: 0.6809 - val_accuracy: 0.4537
Epoch 2/2

Epoch 00002: saving model to model.h5
26/26 - 20s - loss: 0.6405 - accuracy: 0.6667 - val_loss: 0.6263 - val_accuracy: 0.8049


## [Early stopping](https://keras.io/api/callbacks/early_stopping/)

Stop training when a monitored metric has stopped improving.

In [15]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[EarlyStopping(
              patience=3,
              min_delta=0.05,
              baseline=0.8,
              mode='min',
              monitor='val_loss',
              restore_best_weights=True,
              verbose=1)
          ])

Epoch 1/50
26/26 - 20s - loss: 0.6780 - accuracy: 0.5791 - val_loss: 0.6670 - val_accuracy: 0.6439
Epoch 2/50
26/26 - 20s - loss: 0.6466 - accuracy: 0.6521 - val_loss: 0.6281 - val_accuracy: 0.6878
Epoch 3/50
26/26 - 20s - loss: 0.6009 - accuracy: 0.7360 - val_loss: 0.5948 - val_accuracy: 0.7171
Epoch 4/50
26/26 - 20s - loss: 0.5615 - accuracy: 0.7348 - val_loss: 0.6328 - val_accuracy: 0.6049
Epoch 5/50
26/26 - 20s - loss: 0.5348 - accuracy: 0.7409 - val_loss: 0.7063 - val_accuracy: 0.5366
Epoch 6/50
Restoring model weights from the end of the best epoch.
26/26 - 20s - loss: 0.4726 - accuracy: 0.7920 - val_loss: 0.5923 - val_accuracy: 0.6488
Epoch 00006: early stopping


## [CSV Logger](https://keras.io/api/callbacks/csv_logger/)

Callback that streams epoch results to a CSV file.

In [16]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
csv_file = 'training.csv'

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[CSVLogger(csv_file)
          ])

Epoch 1/5
26/26 [==============================] - 21s 824ms/step - loss: 0.6753 - accuracy: 0.5864 - val_loss: 0.6462 - val_accuracy: 0.6976
Epoch 2/5
26/26 [==============================] - 20s 779ms/step - loss: 0.6112 - accuracy: 0.7263 - val_loss: 0.5913 - val_accuracy: 0.7707
Epoch 3/5
26/26 [==============================] - 20s 750ms/step - loss: 0.5567 - accuracy: 0.7676 - val_loss: 0.4934 - val_accuracy: 0.9024
Epoch 4/5
26/26 [==============================] - 20s 773ms/step - loss: 0.4566 - accuracy: 0.8552 - val_loss: 0.4191 - val_accuracy: 0.8683
Epoch 5/5
26/26 [==============================] - 20s 762ms/step - loss: 0.4510 - accuracy: 0.8273 - val_loss: 0.3398 - val_accuracy: 0.9220


In [17]:
pd.read_csv(csv_file).head()

,epoch,accuracy,loss,val_accuracy,val_loss
0,0,0.586375,0.674449,0.697561,0.646243
1,1,0.726277,0.611651,0.770732,0.591342
2,2,0.767640,0.557486,0.902439,0.493365
3,3,0.855231,0.456428,0.868293,0.419099
4,4,0.827251,0.451317,0.921951,0.339797


## [Learning Rate Scheduler](https://keras.io/api/callbacks/learning_rate_scheduler/)

Updates the learning rate during training.

In [18]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
def step_decay(epoch):
	initial_lr = 0.01
	drop = 0.5
	epochs_drop = 1
	lr = initial_lr * math.pow(drop, math.floor((1+epoch)/epochs_drop))
	return lr

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[LearningRateScheduler(step_decay, verbose=1),
                    TensorBoard(log_dir='./log_dir')])


Epoch 00001: LearningRateScheduler reducing learning rate to 0.005.
Epoch 1/5
26/26 [==============================] - 20s 782ms/step - loss: 0.6721 - accuracy: 0.5998 - val_loss: 0.6578 - val_accuracy: 0.6976

Epoch 00002: LearningRateScheduler reducing learning rate to 0.0025.
Epoch 2/5
26/26 [==============================] - 22s 842ms/step - loss: 0.6385 - accuracy: 0.6837 - val_loss: 0.6484 - val_accuracy: 0.6049

Epoch 00003: LearningRateScheduler reducing learning rate to 0.00125.
Epoch 3/5
26/26 [==============================] - 20s 756ms/step - loss: 0.6223 - accuracy: 0.7190 - val_loss: 0.6427 - val_accuracy: 0.6000

Epoch 00004: LearningRateScheduler reducing learning rate to 0.000625.
Epoch 4/5
26/26 [==============================] - 20s 757ms/step - loss: 0.6135 - accuracy: 0.7092 - val_loss: 0.6316 - val_accuracy: 0.6878

Epoch 00005: LearningRateScheduler reducing learning rate to 0.0003125.
Epoch 5/5
26/26 [==============================] - 20s 775ms/step - loss: 0.6

In [19]:
%tensorboard --logdir log_dir

## [ReduceLROnPlateau](https://keras.io/api/callbacks/reduce_lr_on_plateau/)

Reduce learning rate when a metric has stopped improving.

In [ ]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          callbacks=[ReduceLROnPlateau(monitor='val_loss', 
                                       factor=0.2, verbose=1,
                                       patience=1, min_lr=0.001),
                     TensorBoard(log_dir='./log_dir')])

Epoch 1/50
26/26 [==============================] - 22s 837ms/step - loss: 0.6593 - accuracy: 0.6265 - val_loss: 0.6559 - val_accuracy: 0.5268
Epoch 2/50
23/26 [=========================>....] - ETA: 2s - loss: 0.5998 - accuracy: 0.6821

In [ ]:
%tensorboard --logdir log_dir